# Interactively Search CMR for  ABoVE Collections & Granules

https://above.nasa.gov/implementation_plan/standard_projection.html#reference

`![ABoVE Grid AB](https://above.nasa.gov/images/standard_projection_image_20170621.png)`

In [3]:
from ABoVE import *

In [29]:
above_datasets = cmr_search(dict(
    project="ABoVE", 
    page_size="1000"))

above_datasets["items"][0]

{'meta': {'revision-id': 4,
  'deleted': False,
  'format': 'application/echo10+xml',
  'provider-id': 'ORNL_DAAC',
  'user-id': 'ornl_daac',
  'has-formats': False,
  'has-spatial-subsetting': False,
  'native-id': 'A Concise Experiment Plan for the Arctic-Boreal Vulnerability Experiment',
  'has-transforms': False,
  'has-variables': False,
  'concept-id': 'C1536961538-ORNL_DAAC',
  'revision-date': '2019-04-15T18:55:49Z',
  'granule-count': 0,
  'concept-type': 'collection'},
 'umm': {'CollectionCitations': [{'OtherCitationDetails': 'ABOVE SCIENCE DEFINITION TEAM. 2014. A Concise Experiment Plan for the Arctic-Boreal Vulnerability Experiment. ORNL DAAC, Oak Ridge, Tennessee, USA. https://doi.org/10.3334/ORNLDAAC/1617',
    'DOI': {'Authority': 'https://doi.org', 'DOI': '10.3334/ORNLDAAC/1617'}}],
  'SpatialExtent': {'SpatialCoverageType': 'HORIZONTAL',
   'HorizontalSpatialDomain': {'Geometry': {'CoordinateSystem': 'CARTESIAN',
     'BoundingRectangles': [{'WestBoundingCoordinate': 

In [36]:
meta = above_datasets["items"][0]["meta"]
umm = above_datasets["items"][0]["umm"]

meta

{'revision-id': 4,
 'deleted': False,
 'format': 'application/echo10+xml',
 'provider-id': 'ORNL_DAAC',
 'user-id': 'ornl_daac',
 'has-formats': False,
 'has-spatial-subsetting': False,
 'native-id': 'A Concise Experiment Plan for the Arctic-Boreal Vulnerability Experiment',
 'has-transforms': False,
 'has-variables': False,
 'concept-id': 'C1536961538-ORNL_DAAC',
 'revision-date': '2019-04-15T18:55:49Z',
 'granule-count': 0,
 'concept-type': 'collection'}

In [37]:
umm

{'CollectionCitations': [{'OtherCitationDetails': 'ABOVE SCIENCE DEFINITION TEAM. 2014. A Concise Experiment Plan for the Arctic-Boreal Vulnerability Experiment. ORNL DAAC, Oak Ridge, Tennessee, USA. https://doi.org/10.3334/ORNLDAAC/1617',
   'DOI': {'Authority': 'https://doi.org', 'DOI': '10.3334/ORNLDAAC/1617'}}],
 'SpatialExtent': {'SpatialCoverageType': 'HORIZONTAL',
  'HorizontalSpatialDomain': {'Geometry': {'CoordinateSystem': 'CARTESIAN',
    'BoundingRectangles': [{'WestBoundingCoordinate': -176.12,
      'NorthBoundingCoordinate': 81.61,
      'EastBoundingCoordinate': -66.92,
      'SouthBoundingCoordinate': 39.42}]}},
  'GranuleSpatialRepresentation': 'CARTESIAN'},
 'Distributions': [{'Sizes': [{}],
   'DistributionFormat': 'Not provided',
   'Fees': '0'}],
 'CollectionProgress': 'COMPLETE',
 'ScienceKeywords': [{'Category': 'EARTH SCIENCE',
   'Topic': 'ATMOSPHERE',
   'Term': 'ATMOSPHERIC CHEMISTRY',
   'VariableLevel1': 'CARBON AND HYDROCARBON COMPOUNDS',
   'VariableLeve

In [42]:
umm.keys()

dict_keys(['CollectionCitations', 'SpatialExtent', 'Distributions', 'CollectionProgress', 'ScienceKeywords', 'TemporalExtents', 'ProcessingLevel', 'ShortName', 'EntryTitle', 'RelatedUrls', 'ContactGroups', 'DataDates', 'Abstract', 'MetadataDates', 'Version', 'Projects', 'ContactPersons', 'DataCenters', 'Platforms'])

In [58]:
umm["SpatialExtent"]

{'SpatialCoverageType': 'HORIZONTAL',
 'HorizontalSpatialDomain': {'Geometry': {'CoordinateSystem': 'CARTESIAN',
   'BoundingRectangles': [{'WestBoundingCoordinate': -176.12,
     'NorthBoundingCoordinate': 81.61,
     'EastBoundingCoordinate': -66.92,
     'SouthBoundingCoordinate': 39.42}]}},
 'GranuleSpatialRepresentation': 'CARTESIAN'}

In [74]:
try:
    
    # PUBLICATION
    short_name = umm["ShortName"]
    title = umm["EntryTitle"]
    science_keywords = umm["ScienceKeywords"]
    
    citation = umm["CollectionCitations"][0]["OtherCitationDetails"]
    doi = umm["CollectionCitations"][0]["DOI"]["DOI"]
    progress = umm["CollectionProgress"]
    
    urls = umm["RelatedUrls"]
    
    
    # SPATIAL 
    spatial_extent = umm["SpatialExtent"]
    geometry = spatial_extent["HorizontalSpatialDomain"]["Geometry"]
    bounds = geometry["BoundingRectangles"]
    
    min_lon = bounds["WestBoundingCoordinate"]
    max_lon = bounds["EastBoundingCoordinate"]
    min_lat = bounds["SouthBoundingCoordinate"]
    max_lat = bounds["NorthBoundingCoordinate"]
    
    # TEMPORAL
    range_time = umm["TemporalExtents"][0]["RangeDateTimes"]
    start_time = range_time[0]["BeginningDateTime"]
    end_time = range_time[0]["EndingDateTime"]

    

    
    
    

except:
    
    pass

[{'WestBoundingCoordinate': -176.12, 'NorthBoundingCoordinate': 81.61, 'EastBoundingCoordinate': -66.92, 'SouthBoundingCoordinate': 39.42}]


[{'BeginningDateTime': '2014-01-01T00:00:00.000Z',
  'EndingDateTime': '2021-12-31T23:59:59.000Z'}]

In [92]:


umm = above_datasets["items"][20]["umm"]
umm["RelatedUrls"]

[{'Description': 'This link allows direct data access via Earthdata login',
  'URLs': ['https://daac.ornl.gov/daacdata/above/Maps_AGB_North_Slope_AK/'],
  'Relation': ['GET DATA']},
 {'Description': 'ORNL DAAC Data Set Documentation',
  'URLs': ['https://daac.ornl.gov/ABOVE/guides/Maps_AGB_North_Slope_AK.html'],
  'Relation': ['VIEW RELATED INFORMATION', 'GENERAL DOCUMENTATION']},
 {'Description': 'Data set Landing Page DOI URL',
  'URLs': ['https://doi.org/10.3334/ORNLDAAC/1565'],
  'Relation': ['DATA SET LANDING PAGE']},
 {'Description': 'A PDF of the guide ',
  'URLs': ['https://daac.ornl.gov/daacdata/above/Maps_AGB_North_Slope_AK/comp/Maps_AGB_North_Slope_AK.pdf'],
  'Relation': ['VIEW RELATED INFORMATION', 'GENERAL DOCUMENTATION']},
 {'Description': 'A file in comma-separated 9.csv) format of the filed data',
  'URLs': ['https://daac.ornl.gov/daacdata/above/Maps_AGB_North_Slope_AK/comp/Maps_AGB_North_Slope_AK_FieldSites.csv'],
  'Relation': ['VIEW RELATED INFORMATION', 'GENERAL DO

'ABoVE_Concise_Experiment_Plan_1617'

'A Concise Experiment Plan for the Arctic-Boreal Vulnerability Experiment'

['GET DATA']
['VIEW RELATED INFORMATION', 'GENERAL DOCUMENTATION']
['DATA SET LANDING PAGE']
['VIEW RELATED INFORMATION', 'GENERAL DOCUMENTATION']
['VIEW RELATED INFORMATION', 'GENERAL DOCUMENTATION']
['VIEW RELATED INFORMATION', 'GENERAL DOCUMENTATION']
['GET RELATED VISUALIZATION']


In [85]:
umm["RelatedUrls"][-1]

{'Description': 'The Arctic-Boreal Vulnerability Experiment (ABoVE) is a NASA Terrestrial Ecology Program field campaign that will take place in Alaska and western Canada between 2016 and 2021.',
 'URLs': ['https://daac.ornl.gov/ABOVE/guides/ABoVE_Concise_Experiment_Plan_Fig1.png'],
 'Relation': ['GET RELATED VISUALIZATION']}

In [71]:
umm["Platforms"]

[{'Type': 'In Situ Land-based Platforms',
  'ShortName': 'FIELD INVESTIGATION',
  'Instruments': [{'ShortName': 'Computer', 'LongName': 'Computer'}]}]

In [72]:
umm["Abstract"]

"This document presents the Concise Experiment Plan for NASA's Arctic-Boreal Vulnerability Experiment (ABoVE) to serve as a guide to the Program as it identifies the research to be conducted under this study. Research for ABoVE will link field-based, process-level studies with geospatial data products derived from airborne and satellite remote sensing, providing a foundation for improving the analysis and modeling capabilities needed to understand and predict ecosystem responses and societal implications. The ABoVE Concise Experiment Plan (ACEP) outlines the conceptual basis for the Field Campaign and expresses the compelling rationale explaining the scientific and societal importance of the study. It presents both the science questions driving ABoVE research as well as the top-level requirements for a study design to address them."

In [47]:
above_core_datasets = []
for d in above_datasets["items"]:
    if d["umm"]["Projects"][0]["ShortName"]=="ABoVE":
        above_core_datasets.append(d)

print(len(above_core_datasets))

81


In [55]:
for d in above_core_datasets:
    print(d["umm"]["DataCenters"][0]["ShortName"])

ORNL_DAAC
NASA NSIDC DAAC
NASA NSIDC DAAC
ORNL_DAAC
ORNL_DAAC
ORNL_DAAC
ORNL_DAAC
ORNL_DAAC
ORNL_DAAC
ORNL_DAAC
ORNL_DAAC
ORNL_DAAC
ORNL_DAAC
ORNL_DAAC
ORNL_DAAC
ORNL_DAAC
ORNL_DAAC
ORNL_DAAC
ORNL_DAAC
ORNL_DAAC
ORNL_DAAC
ORNL_DAAC
ORNL_DAAC
ORNL_DAAC
ORNL_DAAC
ORNL_DAAC
ORNL_DAAC
ORNL_DAAC
ORNL_DAAC
ORNL_DAAC
ORNL_DAAC
ORNL_DAAC
ORNL_DAAC
ORNL_DAAC
ORNL_DAAC
ORNL_DAAC
ORNL_DAAC
ORNL_DAAC
ORNL_DAAC
ORNL_DAAC
ORNL_DAAC
ORNL_DAAC
ORNL_DAAC
ORNL_DAAC
ORNL_DAAC
ORNL_DAAC
ORNL_DAAC
ORNL_DAAC
ORNL_DAAC
ORNL_DAAC
ORNL_DAAC
ORNL_DAAC
ORNL_DAAC
ORNL_DAAC
ORNL_DAAC
ORNL_DAAC
ORNL_DAAC
ORNL_DAAC
ORNL_DAAC
ORNL_DAAC
ORNL_DAAC
ORNL_DAAC
ORNL_DAAC
ORNL_DAAC
ORNL_DAAC
ORNL_DAAC
ORNL_DAAC
ORNL_DAAC
ORNL_DAAC
ORNL_DAAC
ORNL_DAAC
ORNL_DAAC
ORNL_DAAC
ORNL_DAAC
ORNL_DAAC
ORNL_DAAC
ORNL_DAAC
ORNL_DAAC
ORNL_DAAC
ORNL_DAAC
ORNL_DAAC


In [8]:
def updateme(*args, **kwargs):
    """ """
    print(*args)
    print(**kwargs)

app.tab.on(handler=updateme, names="value")

In [12]:
dir(granules)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_base_url',
 '_build_url',
 '_format',
 '_route',
 '_valid_formats_regex',
 '_valid_state',
 'bounding_box',
 'cloud_cover',
 'day_night_flag',
 'downloadable',
 'entry_title',
 'format',
 'get',
 'get_all',
 'granule_ur',
 'hits',
 'instrument',
 'line',
 'mode',
 'online_only',
 'options',
 'orbit_number',
 'parameters',
 'params',
 'platform',
 'point',
 'polygon',
 'short_name',
 'temporal',
 'version']

In [ ]:
for index, row in app.union.iterrows():
    print(index)
    print(row["boxes"])

In [ ]:
above_results_df

In [ ]:
app.selected_collections.iloc[0]

In [ ]:
for i,r in app.selected_collections.items():
    print(i["datasse"])
    print(r)


In [ ]:
for i, r in app.selected_collections.iterrows():
    print(i)
    print(r)

In [ ]:
#above_datasets = collections.keyword("*ABoVE*").get_all()

#print(len(above_datasets))
print(json.dumps(above_datasets[0], indent=4))

#for d in above_datasets:
#    print(d["dataset_id"])

In [ ]:
app.map.layers[2].layers[0].clear_layers()

In [ ]:
app.union.to_wkt()

In [ ]:
x,y = 

In [ ]:
y

In [ ]:
dir(app.union.exterior.coords)

In [ ]:
from CMR import CollectionQuery, GranuleQuery

collections = CollectionQuery()
granules = GranuleQuery()

In [ ]:
ornlabovedf = query_ornl_projects()
ornlabovedf.head()

In [ ]:
boxes = ornlabovedf.boxes
boxes[:5]

In [ ]:




#text_box = 
#text_box

In [ ]:


shape(test_geojson)
    
    
    
    
   def CMR_box_to_geojson(box, id=0):
    """ """
    
    box = box.split(" ")
    
    ll = [box[1], box[0]]
    lr = [box[3], box[0]]
    ur = [box[3], box[2]]
    ul = [box[1], box[2]]

    polygon = {"type": "Polygon", "coordinates": [ll, lr, ur,ul, ll]}

    return(polygon) 
    

In [ ]:
def geojson_intersect_cmr_box(selection_geojson, cmr_box):
    """ """
    
    selection_geom = shape(selection_geojson)
    cmr_geom = shape(cmr_box)

In [ ]:
print("Number of ABoVE datasets at ORNL DAAC:"+str(len(above)))
print(print(json.dumps(above[0], indent=4)))